In [1]:
import pandas as pd
from pathlib import Path
import json
from pprint import pprint
import math
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm

DATA_DIR = Path("data/benchmarking/nogif")

In [2]:
def get_in_road_percentage(steps_df: pd.DataFrame) -> float:
    ret = steps_df["out_of_road"].value_counts(normalize=True).at[False]
    return ret

In [3]:
def get_n_sidewalk_crashes(steps_df: pd.DataFrame) -> int:
    """Count number of crash episodes to not count same crash multiple times"""
    try:
        # count number of times "crash" becomes True for some time
        n_crashes = steps_df["crash_sidewalk"].diff().value_counts().at[True]

        # need to divide by 2 beacouse diff counts twice
        n_crashes /= 2

        # just in case crash is last episode and we have 3.5 crash episodes make it 4
        n_crashes = math.ceil(n_crashes)
    except KeyError:
        n_crashes = 0

    return n_crashes

In [4]:
def process_steps(steps_infos: list) -> dict:
    """Accepts a list of steps and returns a dict of interesting data"""

    steps_df = pd.DataFrame(steps_infos)
    steps_data = {}
    last_step = steps_df.iloc[-1]

    steps_data = {
        "termination.arrive_dest": last_step["arrive_dest"],
        "termination.timeout": last_step["max_step"],
        "route_completion": last_step["route_completion"],
        "seed": last_step["env_seed"],
        "in_road_percentage": get_in_road_percentage(steps_df),
        "n_sidewalk_crashes": get_n_sidewalk_crashes(steps_df),
    }

    return steps_data

In [5]:
def get_scenarios_df(dir: Path):
    paths = list(dir.glob("*/*.json"))

    scenarios = []
    for file_path in tqdm(paths):

        with open(file_path, "r") as f:
            scenario_data = json.load(f)

        dir = file_path.__str__().split("/")[-2]
        _, dr, _, dt = dir.split("_")
        scenario_data["decision_repeat"] = int(dr)
        scenario_data["dt"] = float(dt)

        steps_infos = scenario_data.pop("steps_infos")
        scenario_data.update(process_steps(steps_infos))
        scenarios.append(scenario_data)
    return pd.DataFrame(scenarios)

In [6]:
df = get_scenarios_df(DATA_DIR)

100%|██████████| 1200/1200 [00:07<00:00, 164.30it/s]


In [7]:
df = df.set_index(
    [
        "dt",
        "decision_repeat",
        "seed",
    ],
    verify_integrity=True,
)

df = df.sort_index()

In [8]:
df

def.map_seq  \
dt   decision_repeat seed                                                      
0.02 5               0     [{'id': 'I', 'pre_block_socket_index': None}, ...   
                     1     [{'id': 'I', 'pre_block_socket_index': None}, ...   
                     2     [{'id': 'I', 'pre_block_socket_index': None}, ...   
                     3     [{'id': 'I', 'pre_block_socket_index': None}, ...   
                     4     [{'id': 'I', 'pre_block_socket_index': None}, ...   
...                                                                      ...   
0.04 20              95    [{'id': 'I', 'pre_block_socket_index': None}, ...   
                     96    [{'id': 'I', 'pre_block_socket_index': None}, ...   
                     97    [{'id': 'I', 'pre_block_socket_index': None}, ...   
                     98    [{'id': 'I', 'pre_block_socket_index': None}, ...   
                     99    [{'id': 'I', 'pre_block_socket_index': None}, ...   

                                                                 def.bv_data  \
dt   decision_repeat seed                                                      
0.02 5               0     [{'length': 4.515, 'width': 1.852, 'height': 1...   
                     1     [{'length': 4.515, 'width': 1.852, 'height': 1...   
                     2     [{'length': 4.515, 'width': 1.852, 'height': 1...   
                     3     [{'length': 4.515, 'width': 1.852, 'height': 1...   
                     4     [{'length': 4.515, 'width': 1.852, 'height': 1...   
...                                                                      ...   
0.04 20              95    [{'length': 4.515, 'width': 1.852, 'height': 1...   
                     96    [{'length': 4.515, 'width': 1.852, 'height': 1...   
                     97    [{'length': 4.515, 'width': 1.852, 'height': 1...   
                     98    [{'length': 4.515, 'width': 1.852, 'height': 1...   
                     99    [{'length': 4.515, 'width': 1.852, 'height': 1...   

                           def.max_steps     start_ts  initialized_ts  \
dt   decision_repeat seed                                               
0.02 5               0              3364  2409.753593     2410.150784   
                     1              1892  2442.576460     2442.807679   
                     2              3574  2456.424512     2456.725837   
                     3              2092  2494.489910     2494.792035   
                     4              2815  2510.568123     2510.821323   
...                                  ...          ...             ...   
0.04 20              95              423  5850.284285     5850.772111   
                     96              330  5898.481414     5898.928606   
                     97              352  5946.949279     5947.688412   
                     98              342  5997.991348     5998.497976   
                     99              251  6023.245874     6023.530236   

                           scenario_done_ts  init_time  scenario_time  \
dt   decision_repeat seed                                               
0.02 5               0          2415.246879   0.397191       5.096095   
                     1          2446.031881   0.231219       3.224202   
                     2          2460.962125   0.301324       4.236289   
                     3          2496.538788   0.302125       1.746753   
                     4          2513.785923   0.253200       2.964600   
...                                     ...        ...            ...   
0.04 20              95         5851.379656   0.487826       0.607545   
                     96         5900.174167   0.447192       1.245561   
                     97         5947.881506   0.739133       0.193094   
                     98         6000.163613   0.506628       1.665637   
                     99         6023.934485   0.284362       0.404249   

                           total_time  n_crashed_vehicles  \
dt   decision_repeat s

In [9]:
# calculate driving score
# ! Problem changing the values here in the analysis can change % error
def calculate_driving_score(df):
    df["driving_score"] = (
        df["route_completion"]
        * df["in_road_percentage"]
        * (0.65) ** df["n_sidewalk_crashes"]
        * (0.60) ** df["n_crashed_vehicles"]
    )
    return df


df = calculate_driving_score(df)

In [10]:
df["driving_score"].describe()

count    1200.000000
mean        0.557781
std         0.332933
min         0.001882
25%         0.265636
50%         0.594773
75%         0.990249
max         1.000638
Name: driving_score, dtype: float64

# Getting featuers


## Getting scenario featuers


In [11]:
def get_features(col_name):
    x = df[col_name]
    x = x.reset_index(drop=True)
    x = pd.json_normalize(x)
    x = pd.concat(
        [pd.json_normalize(x[col]).add_prefix(f"{col}.") for col in x], axis=1
    )
    # convert lists to strings
    x = x.map(lambda x: ",".join(x) if isinstance(x, list) else x)
    x = pd.get_dummies(x)
    x = x.add_prefix(f"{col_name}.")
    return x

In [12]:
def_columns = ["def.map_seq", "def.bv_data"]


map_feat = get_features("def.map_seq")
bv_feat = get_features("def.bv_data")

S = map_feat.join(bv_feat).to_numpy()

In [13]:
S.shape

(1200, 1309)

## Get fid vector


In [14]:
import numpy as np

fid = df.index.droplevel("seed")
fid = fid.to_numpy()

fid = np.vstack(fid)

fid.shape, fid,

((1200, 2),
 array([[ 0.02,  5.  ],
        [ 0.02,  5.  ],
        [ 0.02,  5.  ],
        ...,
        [ 0.04, 20.  ],
        [ 0.04, 20.  ],
        [ 0.04, 20.  ]]))

### Adding to scenario data


In [15]:
X = np.hstack((fid, S))
X.shape

(1200, 1311)

## Get DScore vector


In [16]:
y = df["driving_score"].to_numpy()
y.shape, y

((1200,),
 array([0.99316503, 0.12507196, 0.99343402, ..., 0.08054419, 0.21511478,
        0.38666393]))

# Fitting GPR


### Handling NaNs


In [17]:
X[pd.isna(X)] = -1
X

array([[0.02, 5.0, 75.58282470703125, ..., False, False, False],
       [0.02, 5.0, -1, ..., False, False, False],
       [0.02, 5.0, -1, ..., False, False, False],
       ...,
       [0.04, 20.0, -1, ..., False, False, False],
       [0.04, 20.0, -1, ..., False, False, False],
       [0.04, 20.0, -1, ..., False, False, False]], dtype=object)

In [18]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import r2_score

## LOO testing


In [19]:
loo = LeaveOneOut()

loo.get_n_splits(X)

1200

In [20]:
n_tests = 100

ys = []

for i, (train_index, test_index) in enumerate(loo.split(X)):
    # print(train_index)
    # print(test_index)
    kernel = RBF(length_scale_bounds=(1e-5, 1e5))
    gaussian_process = GaussianProcessRegressor(
        kernel=kernel, n_restarts_optimizer=9, random_state=2137
    )

    gaussian_process.fit(X[train_index], y[train_index])

    y_pred, y_std = gaussian_process.predict(X[test_index], return_std=True)
    y_true = y[test_index]
    print(
        f"For index: {test_index} we predicted {y_pred} +- {y_std} but True value is: {y_true}"
    )

    ys.append([y_true, y_pred])

    # if i == n_tests:
    #     break

For index: [0] we predicted [1.36140679] +- [0.39534716] but True value is: [0.99316503]
For index: [1] we predicted [-0.18634378] +- [0.39575025] but True value is: [0.12507196]
For index: [2] we predicted [1.29209037] +- [0.39552411] but True value is: [0.99343402]
For index: [3] we predicted [0.71148503] +- [0.39599083] but True value is: [0.98920761]
For index: [4] we predicted [0.42812767] +- [0.39598992] but True value is: [0.5945412]
For index: [5] we predicted [0.59392097] +- [0.39598985] but True value is: [0.59565215]
For index: [6] we predicted [0.71232632] +- [0.39599112] but True value is: [0.98964735]
For index: [7] we predicted [0.63684988] +- [0.39560103] but True value is: [0.2899516]
For index: [8] we predicted [0.25585778] +- [0.39584423] but True value is: [0.59333825]
For index: [9] we predicted [0.71496227] +- [0.39557391] but True value is: [0.35753353]
For index: [10] we predicted [0.71445786] +- [0.39599092] but True value is: [0.99320721]
For index: [11] we pr

In [ ]:
ys = np.array(ys).squeeze()
y_true = ys.T[0]
y_preds = ys.T[1]
y_preds

In [ ]:
r2_score(y_true, y_preds)

In [ ]:
y_preds_cliped = np.clip(y_preds, 0, 1)
r2_score(y_true, y_preds_cliped)